In [1]:
import tensorflow as tf

print(f'tensorflow version {tf.__version__}')

from imblearn.over_sampling import SMOTE
from collections import Counter
from numpy import * 

import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

tensorflow version 2.9.1


# Loading the Data & Smote

In [2]:
def dataLoadSm():
    test = pd.read_csv('Real_Tomorrow/test_real_tom_target.csv')
    train = pd.read_csv('Real_Tomorrow/training_real_tom_target.csv')
    val = pd.read_csv('Real_Tomorrow/validation_real_tom_target.csv')
    print(train.head())
    
    #Split data
    x_train = train[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******training features******")
    print (x_train)
    y_train = train[['Real_tom_lsm']]
    print ("******training target******")
    print (y_train)
    
    x_test = test[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******test features******")
    print (x_test)
    y_test = test[['Real_tom_lsm']]
    print ("******test target******")
    print (y_test)
    
    x_val = val[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******validation features******")
    print (x_val)
    y_val = val[['Real_tom_lsm']]
    print ("******validation target******")
    print (y_val)
    
    print('Before Smote')

    dff = y_train
    M = y_train.to_numpy()
    # summarize dataset
    classes = unique(M)
    print(classes)
    total = len(M)
    for c in classes:
        n_examples = len(M[M==c])
        percent = n_examples / total * 100
        print('> Class=%d : %d/%d (%.1f%%)' % (c, n_examples, total, percent))
    
    
    smt = SMOTE()
    
    X_train_sm, Y_train_sm = smt.fit_resample(x_train, y_train)
    
    print('After SMOTE')
    dff = Y_train_sm
    M = Y_train_sm.to_numpy()
    # summarize dataset
    classes = unique(M)
    print(classes)
    total = len(M)
    for c in classes:
        n_examples = len(M[M==c])
        percent = n_examples / total * 100
        print('> Class=%d : %d/%d (%.1f%%)' % (c, n_examples, total, percent))
        
    return X_train_sm, Y_train_sm, x_val, y_val, x_test, y_test

def dataLoad():
    test = pd.read_csv('Real_Tomorrow/test_real_tom_target.csv')
    train = pd.read_csv('Real_Tomorrow/training_real_tom_target.csv')
    val = pd.read_csv('Real_Tomorrow/validation_real_tom_target.csv')
    print(train.head())
    
    #Split data
    x_train = train[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******training features******")
    #print (x_train)
    y_train = train[['Real_tom_lsm']]
    print ("******training target******")
    #print (y_train)
    
    x_test = test[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******test features******")
    #print (x_test)
    y_test = test[['Real_tom_lsm']]
    print ("******test target******")
    #print (y_test)
    
    x_val = val[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******validation features******")
    #print (x_val)
    y_val = val[['Real_tom_lsm']]
    print ("******validation target******")
    #print (y_val)
    
        
    return x_train, y_train, x_val, y_val, x_test, y_test

In [3]:
x_train_sm, y_train_sm, x_val, y_val, x_test, y_test = dataLoadSm()
x_train, y_train, x_val, y_val, x_test, y_test = dataLoad()


   Unnamed: 0        time  latitude  longitude        vo          r     u_200  \
0           0  1980-01-01       0.0       20.0  0.000007  80.761185  1.909660   
1           1  1980-01-01       0.0       22.5  0.000004  80.703650  1.165733   
2           2  1980-01-01       0.0       25.0  0.000007  78.231514 -1.311676   
3           3  1980-01-01       0.0       27.5  0.000010  79.631010 -3.777573   
4           4  1980-01-01       0.0       30.0  0.000010  71.573875 -5.734505   

      u_850     v_200     v_850        ttr  sst  lsm  Real_tom_lsm  
0 -3.323872  1.687164 -1.823624 -247.54074  0.0  0.0           0.0  
1 -2.844494  1.060593 -1.991425 -240.00592  0.0  0.0           0.0  
2 -2.125244  3.280617 -1.931789 -223.76889  0.0  0.0           0.0  
3 -1.122395  5.743889 -1.243538 -235.55556  0.0  0.0           0.0  
4 -1.362953  6.514030 -0.954163 -254.03260  0.0  0.0           0.0  
******training features******
               vo          r      u_200     u_850      v_200     v_85

# Standardization

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

#scaled_x_training = scaler.fit_transform(x_training)
#df = pd.DataFrame(StandardScaler().fit_transform(x_training))

X_train_stand = x_train_sm.copy() #smote
X_valid_stand = x_val.copy() #smote
X_test_stand = x_test.copy() #smote

#not smote
X_train_scaled = x_train.copy() 
X_valid_scaled = x_val.copy()
X_test_scaled = x_test.copy()

num_cols = [ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']

# apply standardization on numerical features
for i in num_cols:
    
    # fit on training data column
    scale = StandardScaler().fit(X_train_stand[[i]])
    
    # transform the training data column
    X_train_stand[i] = scale.transform(X_train_stand[[i]])
    
    # transform the training data column
    X_valid_stand[i] = scale.transform(X_valid_stand[[i]])   

    # transform the testing data column
    X_test_stand[i] = scale.transform(X_test_stand[[i]])
    
for i in num_cols:
    
    # fit on training data column
    scale = StandardScaler().fit(X_train_scaled[[i]])
    
    # transform the training data column
    X_train_scaled[i] = scale.transform(X_train_scaled[[i]])
    
    # transform the training data column
    X_valid_scaled[i] = scale.transform(X_valid_scaled[[i]])   

    # transform the testing data column
    X_test_scaled[i] = scale.transform(X_test_scaled [[i]])

# ExtraTreesClassifier

In [21]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import log_loss
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix

# get a list of models to evaluate
def get_models():
    models = dict()
    n_trees = [10, 50, 100, 500, 1000, 5000]
    min_samples_leaf = [1, 10, 50, 100, 500, 1000, 5000]
    max_features= ['sqrt', 'log2', None] 
    for n in n_trees:
        for l in min_samples_leaf:
            for m in max_features:
                models[str(n)+'trees_'+str(l)+'leafSamples_'+str(m)+'maxFeat'] = ExtraTreesClassifier(criterion='log_loss', n_estimators=n, min_samples_leaf=l, max_features=m)
    return models

#ExtraTreesClassifier_class_weight
def get_models_class_weight(method):
    models = dict()
    n_trees = [10, 50, 100, 500, 1000, 5000]
    min_samples_leaf = [1, 10, 50, 100, 500, 1000, 5000]
    max_features= ['sqrt', 'log2', None] 
    
    if method == 'balanced':
        print("The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y))")
        for n in n_trees:
        for l in min_samples_leaf:
            for m in max_features:
                models[str(n)+'trees_'+str(l)+'leafSamples_'+str(m)+'maxFeat'] = ExtraTreesClassifier(criterion='log_loss', n_estimators=n, min_samples_leaf=l, max_features=m, class_weight="balanced")
    elif method == 'balanced_subsample':
        print("The “balanced_subsample” mode is the same as “balanced” except that weights are computed based on the bootstrap sample for every tree grown.")
        for n in n_trees:
        for l in min_samples_leaf:
            for m in max_features:
                models[str(n)+'trees_'+str(l)+'leafSamples_'+str(m)+'maxFeat'] = ExtraTreesClassifier(criterion='log_loss', n_estimators=n, min_samples_leaf=l, max_features=m, class_weight="balanced_subsample")
    else:
        print('the method is not callable')
    return models
# evaluate a given model using validation
def evaluate_model(model, x_train, y_train, x_val, y_val):
    trained_mod = model.fit(x_train, y_train)
    score = log_loss(y_val, model.predict_proba(x_val))
    conf = confusion_matrix(y_val, np.round(model.predict(x_val)))
    np.set_printoptions(precision=2)
    print(conf)  
    return score

# models


In [22]:
models = get_models()


# random_forest_smote

In [ ]:
results, names = list(), list()
for name, model in models.items():
    score = evaluate_model(model, X_train_stand, y_train_sm.values.reshape(-1,), X_valid_stand, y_val.values.reshape(-1,))

    results.append(score)
    names.append(name)

[[685156   2007]
 [   688    551]]
[[685141   2022]
 [   702    537]]
[[685130   2033]
 [   702    537]]
[[666563  20600]
 [   256    983]]
[[672805  14358]
 [   290    949]]
[[677411   9752]
 [   361    878]]
[[643813  43350]
 [   166   1073]]
[[658143  29020]
 [   185   1054]]
[[665353  21810]
 [   226   1013]]
[[641443  45720]
 [   152   1087]]
[[651627  35536]
 [   157   1082]]
[[658499  28664]
 [   178   1061]]
[[620232  66931]
 [   137   1102]]
[[634127  53036]
 [   143   1096]]
[[641803  45360]
 [   139   1100]]
[[613746  73417]
 [   134   1105]]
[[631884  55279]
 [   152   1087]]
[[635871  51292]
 [   134   1105]]
[[588180  98983]
 [   163   1076]]
[[600488  86675]
 [   152   1087]]
[[625320  61843]
 [   122   1117]]
[[685265   1898]
 [   639    600]]
[[685302   1861]
 [   668    571]]
[[685183   1980]
 [   676    563]]
[[669971  17192]
 [   249    990]]


# ExtraTreesClassifier_class_weight

In [19]:
logistic_regression_with_val(X_train_scaled,y_train,X_valid_scaled,y_val)

##########################Penalty:l1, solver:newton-cg, C100##########################
##########################Penalty:l1, solver:lbfgs, C100##########################
##########################Penalty:l1, solver:liblinear, C100##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-0.41705726  0.28400323  0.14547696 -0.68773563 -0.03177079  0.07213172
   0.88752071  0.67371025]]
[-7.5923385]
##########################Penalty:l1, solver:sag, C100##########################
##########################Penalty:l1, solver:saga, C100##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-0.4170667   0.28398066  0.14547838 -0.68774149 -0.03177496  0.07213486
   0.88754934  0.67370835]]
[-7.59234049]
##########################Penalty:l2, solver:newton-cg, C100##########################


...

[[-0.41705441  0.28396706  0.14552268 -0.68776078 -0.03170306  0.07217815
   0.88755953  0.67370496]]
[-7.59236383]
##########################Penalty:none, solver:liblinear, C0.01##########################
##########################Penalty:none, solver:sag, C0.01##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-0.41706423  0.28396452  0.14548065 -0.68771896 -0.03177032  0.07215606
   0.88755692  0.67367951]]
[-7.59229183]
##########################Penalty:none, solver:saga, C0.01##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-0.41706668  0.28398474  0.14548269 -0.68774176 -0.03177587  0.07213654
   0.88754942  0.67371242]]
[-7.59234469]
